<a href="https://colab.research.google.com/github/chaitra0312/ML/blob/main/wasteclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving archive (1) (1).zip to archive (1) (1).zip


In [2]:
!unzip "archive (1) (1).zip" -d ./dataset


Streaming output truncated to the last 5000 lines.
  inflating: ./dataset/train/labels/000055_JPG_jpg.rf.646de53e252df59ecee96c58d045fafd.txt  
  inflating: ./dataset/train/labels/000055_JPG_jpg.rf.6c0cebc42950c9485c1fec799f79d30e.txt  
  inflating: ./dataset/train/labels/000055_JPG_jpg.rf.971eab1d0640140f0b61357f49894103.txt  
  inflating: ./dataset/train/labels/000055_JPG_jpg.rf.9fe266cb29fc471e50a3113363781d45.txt  
  inflating: ./dataset/train/labels/000055_JPG_jpg.rf.a0ffded1b4df9285e860a02c3451e0b3.txt  
  inflating: ./dataset/train/labels/000055_JPG_jpg.rf.afd2c85571e873f9f67b964f56ff44c2.txt  
  inflating: ./dataset/train/labels/000055_JPG_jpg.rf.f8c66618a413ef0343b5170d12a09784.txt  
  inflating: ./dataset/train/labels/000055_JPG_jpg.rf.fbc18cc7703a8884f77cb2797ff8fda5.txt  
  inflating: ./dataset/train/labels/000055_JPG_jpg.rf.ff45590651469ae1234a48aecbb88111.txt  
  inflating: ./dataset/train/labels/000055_jpg.rf.0351831ddeffd9976bd5c2cf6412653f.txt  
  inflating: ./dataset/

In [5]:
!ls ./dataset


data.yaml  README.dataset.txt  README.roboflow.txt  test  train  valid


In [6]:
!ls ./dataset/train


images	labels


In [7]:
import os
import cv2

# YOLO class names from your YAML
class_names = ['Aluminium foil', 'Bottle cap', 'Bottle', 'Broken glass', 'Can',
               'Carton', 'Cigarette', 'Cup', 'Lid', 'Other litter', 'Other plastic',
               'Paper', 'Plastic bag - wrapper', 'Plastic container', 'Pop tab',
               'Straw', 'Styrofoam piece', 'Unlabeled litter']

# Input dataset root (unzipped folder)
dataset_root = "./dataset"
# Output folder for classification dataset
output_root = "./classification_dataset"
os.makedirs(output_root, exist_ok=True)

# Function to crop objects
def process_split(split):
    img_dir = os.path.join(dataset_root, split, "images")
    lbl_dir = os.path.join(dataset_root, split, "labels")
    out_dir = os.path.join(output_root, split)
    os.makedirs(out_dir, exist_ok=True)

    for label_file in os.listdir(lbl_dir):
        if not label_file.endswith(".txt"):
            continue

        image_file = label_file.replace(".txt", ".jpg")
        image_path = os.path.join(img_dir, image_file)

        # sometimes images may be .png
        if not os.path.exists(image_path):
            image_file = label_file.replace(".txt", ".png")
            image_path = os.path.join(img_dir, image_file)

        image = cv2.imread(image_path)
        if image is None:
            continue

        h, w, _ = image.shape

        with open(os.path.join(lbl_dir, label_file), "r") as f:
            for i, line in enumerate(f):
                parts = line.strip().split()
                if len(parts) != 5:
                    continue

                class_id, x_center, y_center, width, height = map(float, parts)
                class_id = int(class_id)

                # convert to pixel coordinates
                x_center *= w
                y_center *= h
                width *= w
                height *= h

                x1 = int(x_center - width / 2)
                y1 = int(y_center - height / 2)
                x2 = int(x_center + width / 2)
                y2 = int(y_center + height / 2)

                # crop and save
                crop = image[max(0,y1):min(h,y2), max(0,x1):min(w,x2)]
                if crop.size == 0:  # skip empty crops
                    continue

                class_name = class_names[class_id]
                save_dir = os.path.join(out_dir, class_name)
                os.makedirs(save_dir, exist_ok=True)

                save_path = os.path.join(save_dir, f"{image_file}_{i}.jpg")
                cv2.imwrite(save_path, crop)

    print(f"Finished processing {split}")

# Process train, valid, test
for split in ["train", "valid", "test"]:
    process_split(split)


Finished processing train
Finished processing valid
Finished processing test


In [8]:
import os
from collections import Counter

def count_images(split):
    split_path = os.path.join("./classification_dataset", split)
    counts = {}
    for class_name in os.listdir(split_path):
        class_dir = os.path.join(split_path, class_name)
        if os.path.isdir(class_dir):
            counts[class_name] = len(os.listdir(class_dir))
    return counts

train_counts = count_images("train")
valid_counts = count_images("valid")
test_counts = count_images("test")

print("Train:", train_counts)
print("Valid:", valid_counts)
print("Test:", test_counts)


Train: {'Broken glass': 376, 'Unlabeled litter': 1419, 'Cup': 583, 'Paper': 369, 'Bottle cap': 1318, 'Cigarette': 2214, 'Plastic container': 145, 'Can': 714, 'Pop tab': 225, 'Bottle': 821, 'Other litter': 356, 'Lid': 252, 'Plastic bag - wrapper': 2228, 'Aluminium foil': 142, 'Styrofoam piece': 304, 'Other plastic': 751, 'Carton': 662, 'Straw': 371}
Valid: {'Broken glass': 123, 'Unlabeled litter': 569, 'Cup': 186, 'Paper': 178, 'Bottle cap': 459, 'Cigarette': 565, 'Plastic container': 90, 'Can': 267, 'Pop tab': 125, 'Bottle': 320, 'Other litter': 178, 'Lid': 93, 'Plastic bag - wrapper': 854, 'Aluminium foil': 62, 'Styrofoam piece': 113, 'Other plastic': 265, 'Carton': 263, 'Straw': 120}
Test: {'Broken glass': 51, 'Unlabeled litter': 38, 'Cup': 19, 'Paper': 10, 'Bottle cap': 26, 'Cigarette': 55, 'Plastic container': 2, 'Can': 19, 'Pop tab': 9, 'Bottle': 13, 'Other litter': 7, 'Lid': 4, 'Plastic bag - wrapper': 59, 'Aluminium foil': 6, 'Styrofoam piece': 6, 'Other plastic': 28, 'Carton': 

In [9]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2

# Load datasets
train_ds = tf.keras.utils.image_dataset_from_directory(
    "/content/classification_dataset/train",
    image_size=(224, 224),
    batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "/content/classification_dataset/valid",
    image_size=(224, 224),
    batch_size=32
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    "/content/classification_dataset/test",
    image_size=(224, 224),
    batch_size=32
)

# ✅ Get class names here before normalization
class_names = train_ds.class_names
num_classes = len(class_names)
print("Classes:", class_names)

# Normalize (scales pixels to [0,1])
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds   = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds  = test_ds.map(lambda x, y: (normalization_layer(x), y))

# Use MobileNetV2 as feature extractor
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(num_classes, activation="softmax")
])

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])


Found 13250 files belonging to 18 classes.
Found 4830 files belonging to 18 classes.
Found 394 files belonging to 18 classes.
Classes: ['Aluminium foil', 'Bottle', 'Bottle cap', 'Broken glass', 'Can', 'Carton', 'Cigarette', 'Cup', 'Lid', 'Other litter', 'Other plastic', 'Paper', 'Plastic bag - wrapper', 'Plastic container', 'Pop tab', 'Straw', 'Styrofoam piece', 'Unlabeled litter']
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [14]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20
)

Epoch 1/20
415/415 ━━━━━━━━━━━━━━━━━━━━ 17s 40ms/step - accuracy: 0.6202 - loss: 1.2133 - val_accuracy: 0.4188 - val_loss: 1.9617
Epoch 2/20
415/415 ━━━━━━━━━━━━━━━━━━━━ 16s 37ms/step - accuracy: 0.6267 - loss: 1.1843 - val_accuracy: 0.4118 - val_loss: 2.0183
Epoch 3/20
415/415 ━━━━━━━━━━━━━━━━━━━━ 17s 40ms/step - accuracy: 0.6346 - loss: 1.1679 - val_accuracy: 0.4083 - val_loss: 2.0282
Epoch 4/20
415/415 ━━━━━━━━━━━━━━━━━━━━ 15s 37ms/step - accuracy: 0.6381 - loss: 1.1475 - val_accuracy: 0.4054 - val_loss: 2.0386
Epoch 5/20
415/415 ━━━━━━━━━━━━━━━━━━━━ 17s 41ms/step - accuracy: 0.6423 - loss: 1.1333 - val_accuracy: 0.3969 - val_loss: 2.0704
Epoch 6/20
415/415 ━━━━━━━━━━━━━━━━━━━━ 16s 38ms/step - accuracy: 0.6468 - loss: 1.1226 - val_accuracy: 0.4091 - val_loss: 2.0822
Epoch 7/20
415/415 ━━━━━━━━━━━━━━━━━━━━ 20s 37ms/step - accuracy: 0.6482 - loss: 1.1089 - val_accuracy: 0.4168 - val_loss: 2.0660
Epoch 8/20
415/415 ━━━━━━━━━━━━━━━━━━━━ 20s 37ms/step - accuracy: 0.6452 - loss: 1.1047 - 

In [15]:
test_loss, test_acc = model.evaluate(test_ds)
print("Test accuracy:", test_acc)


13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.5372 - loss: 1.4859
Test accuracy: 0.5355330109596252


In [24]:
model.save("waste_model.keras")


In [19]:
import os

# list first 10 files inside test
for root, dirs, files in os.walk("test"):
    for name in files[:10]:
        print(os.path.join(root, name))


In [22]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the saved model
model = tf.keras.models.load_model("waste_classifier.keras")

# Pick one sample image from your dataset
img_path = "/content/classification_dataset/test/Bottle cap/000000_jpg.rf.1b029d4c742673020fa91367070f5f51.jpg_0.jpg"
# Preprocess the image
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0  # normalize

# Predict
pred = model.predict(img_array)
class_names = ['Aluminium foil', 'Bottle', 'Bottle cap', 'Broken glass', 'Can', 'Carton', 'Cigarette',
               'Cup', 'Lid', 'Other litter', 'Other plastic', 'Paper', 'Plastic bag - wrapper',
               'Plastic container', 'Pop tab', 'Straw', 'Styrofoam piece', 'Unlabeled litter']

print("Predicted class:", class_names[np.argmax(pred)])


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Predicted class: Straw


In [11]:
import tensorflow as tf
print("GPU Available:", tf.config.list_physical_devices('GPU'))


GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [25]:
from flask import Flask, request, jsonify
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

app = Flask(__name__)

# Load your model
model = load_model("waste_model.keras")
class_names = ['Aluminium foil', 'Bottle', 'Bottle cap', 'Broken glass', 'Can',
               'Carton', 'Cigarette', 'Cup', 'Lid', 'Other litter',
               'Other plastic', 'Paper', 'Plastic bag - wrapper',
               'Plastic container', 'Pop tab', 'Straw', 'Styrofoam piece',
               'Unlabeled litter']

@app.route("/predict", methods=["POST"])
def predict():
    file = request.files["file"]   # Flutter will send image as 'file'
    img = image.load_img(file, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    predictions = model.predict(img_array)
    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = float(np.max(predictions[0]))

    return jsonify({
        "class": predicted_class,
        "confidence": confidence
    })

if __name__ == "__main__":
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (inotify)


In [26]:
from google.colab import files
files.download("waste_model.keras")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>